## Importing all the libraries

In [3]:
import tensorflow as tf
from tensorflow import keras 
from keras import Sequential
from keras.layers import Dense,Conv2D

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings as war
war.filterwarnings('ignore')


In [10]:
count = 0

print(f'Hello world count =  {count}')

Hello world count =  0
